In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.6 MB/s eta 0:00:00


In [2]:
import transformers
transformers.__version__

'4.33.1'

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
!wget -O ./ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O ./ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2023-09-08 07:37:19--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.202.51, 52.219.144.42, 52.219.148.38, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.202.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘./ratings_train.txt’

./ratings_train.txt 100%[===================>]  13.95M  9.18MB/s    in 1.5s    

2023-09-08 07:37:21 (9.18 MB/s) - ‘./ratings_train.txt’ saved [14628807/14628807]

--2023-09-08 07:37:21--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.202.51, 52.219.144.42, 52.219.148.38, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.202.51|:80... connected.
HTTP

In [5]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [6]:
train_data.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [7]:
train_data = train_data.dropna(how = 'any')
train_data = train_data.reset_index(drop = True)
print(train_data.isnull().values.any())

False


In [8]:
test_data = test_data.dropna(how = 'any')
test_data = test_data.reset_index(drop = True)
print(test_data.isnull().values.any())

False


In [9]:
print(len(train_data))
print(len(test_data))

149995
49997


In [10]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',
                                          bos_token = '</s>',
                                          eos_token = '</s>',
                                          pad_token = '<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
print(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [12]:
print(tokenizer.tokenize('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [13]:
tokenizer.decode(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [14]:
tokenizer.decode([11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031])

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [15]:
max_seq_len = 128

In [16]:
encoded_result = tokenizer.encode('전율을 일으키는 영화. 다시보고 싶은 영화',
                                  max_length = max_seq_len,
                                  pad_to_max_length = True)
print(encoded_result)
print('길이: ', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9034, 13555, 16447, 10584, 389, 9427, 11213, 22386, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이:  128


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
tokenizer.bos_token

'</s>'

In [18]:
tokenizer.eos_token

'</s>'

In [19]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

  input_ids, data_labels = [], []

  for example, label in tqdm(zip(examples, labels), total = len(examples)):

    bos_token = [tokenizer.bos_token]
    eos_token = [tokenizer.eos_token]
    tokens = bos_token + tokenizer.tokenize(example) + eos_token
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_id = pad_sequences([input_id], maxlen = max_seq_len,
                             value = tokenizer.pad_token_id,
                             padding = 'post')[0]

    assert len(input_id) == max_seq_len, 'Error with input length {} vs []'.format(len(input_id),
                                                                                   max_seq_len)
    input_ids.append(input_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype = int)
  data_labels = np.asarray(data_labels, dtype = np.int32)

  return input_ids, data_labels

In [20]:
train_X, train_y = convert_examples_to_features(train_data['document'],
                                                train_data['label'],
                                                max_seq_len = max_seq_len,
                                                tokenizer = tokenizer)

test_X, test_y = convert_examples_to_features(train_data['document'],
                                                train_data['label'],
                                                max_seq_len = max_seq_len,
                                                tokenizer = tokenizer)

  0%|          | 0/149995 [00:00<?, ?it/s]

  0%|          | 0/149995 [00:00<?, ?it/s]

In [21]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩: ', input_id)
print('각 인코딩의 길이: ', len(input_id))
print('정수 인코딩 복원: ', tokenizer.decode(input_id))
print('레이블: ', label)

단어에 대한 정수 인코딩:  [    1  9050  9267  7700  9705 23971 12870  8262  7055  7098  8084 48213
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이:  128
정수 인코딩 복원:  </s> 아 더빙.. 진짜 짜증나네요 목소리</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [22]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [23]:
model.summary()

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 125164032 
 r)                                                              
                                                                 
Total params: 125,164,032
Trainable params: 125,164,032
Non-trainable params: 0
_________________________________________________________________


In [24]:
input_ids_layer = tf.keras.layers.Input(shape = (max_seq_len, ), dtype = tf.int32)
outputs = model([input_ids_layer])
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [25]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name):
    super(TFGPT2ForSequenceClassification, self).__init__()
    self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt = True)
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.classifier = tf.keras.layers.Dense(2,
                                            kernel_initializer = \
                                            tf.keras.initializers.TruncatedNormal(0.02),
                                            activation = 'softmax',
                                            name = 'classifier')

  def call(self, inputs):
    outputs = self.gpt(input_ids = inputs)
    cls_token = outputs[0][:, -1]
    cls_token = self.dropout(cls_token)
    prediction = self.classifier(cls_token)

    return prediction

In [26]:
model = TFGPT2ForSequenceClassification('skt/kogpt2-base-v2')
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
model.fit(train_X, train_y, epochs = 1, batch_size = 32, validation_split = 0.2)

3750/3750 [==============================] - 3639s 959ms/step - loss: 0.4077 - accuracy: 0.8216 - val_loss: 0.3058 - val_accuracy: 0.8694


In [ ]:
results = model.evaluate(test_X, test_y, batch_size = 1024)
print('test loss, test acc: ', results)

In [ ]:
def sentiment_predict(new_sentence):
  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen = max_seq_len, value = tokenizer.pad_token_id,
                           padding = 'post')[0]
  input_id = np.array([input_id])
  score = np.argmax(model.predict(input_id))

  if(score == 1):
    print(f'{new_sentence}는 긍정 리뷰입니다.')
  else:
    print(f'{new_sentence}는 부정 리뷰입니다.')